<a href="https://colab.research.google.com/github/mehtasaurav/Concept-Drift-SDP/blob/main/Frouros_Data_drift.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [3]:
!pip install frouros

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#MMD Detector

In [4]:
from frouros.callbacks.batch import PermutationTestOnBatchData
from frouros.detectors.data_drift import MMD

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [48]:
np.random.seed(seed=31)
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/SDP/IoT_2020_b_0.01_fs.csv")
X = df.drop(['Label',],axis=1)
y = df['Label']
(
    X_train,
    X_test,
    y_train,
    y_test,
) = train_test_split(X, y, train_size=0.7, random_state=31)


In [7]:
alpha = 0.01
import numpy as np

# Convert X_train to a NumPy array
X_train_np = np.array(X_train)
X_test_np = np.array(X_test)


In [8]:
print(X.shape)

(6252, 6)


In [9]:
# Fit the detector on the NumPy array
detector = MMD(
    callbacks=[
        PermutationTestOnBatchData(
            num_permutations=1000,
            random_state=31,
            num_jobs=-1,
            name="permutation_test",
            verbose=False,
        ),
    ],
)
_ = detector.fit(X=X_train_np)


In [10]:
pipeline = Pipeline(
    steps=[
        ("scale", StandardScaler()),
        ("model", LogisticRegression(random_state=31)),
    ],
)
pipeline.fit(X=X_train, y=y_train)

Pipeline(steps=[('scale', StandardScaler()),
                ('model', LogisticRegression(random_state=31))])

In [11]:
y_pred = pipeline.predict(X=X_test_np)
p_value = detector.compare(X=X_test_np)[1]["permutation_test"]["p_value"]
print(f"p-value: {round(p_value, 4)}")
if p_value < alpha:
    print("Data drift detected")
else:
    print("No data drift detected")
print(f"Accuracy: {round(accuracy_score(y_test, y_pred), 4)}")

p-value: 0.853
No data drift detected
Accuracy: 1.0


In [12]:
scale_factor = 6.0 # Increase the scale factor to increase the noise
X_test_noise = X_test + np.random.normal(loc=0, scale=scale_factor*X_test.std(axis=0), size=X_test.shape) 
y_pred = pipeline.predict(X=X_test_noise)
p_value = detector.compare(X=X_test_noise)[1]["permutation_test"]["p_value"]
print(f"p-value: {round(p_value, 4)}")
if p_value < alpha:
    print("Data drift detected")
else:
    print("No data drift detected")
print(f"Accuracy: {round(accuracy_score(y_test, y_pred), 4)}")

p-value: 0.0
Data drift detected
Accuracy: 0.6525
